# TOA conversion of provided data

## Unzipping zipped folders

In [ ]:
folder_zips = r"<-- INSERT PATH HERE -->"

In [ ]:
# import OS
import os
for x in os.listdir(folder_zips):
    if x.endswith(".zip"):
        print(x)

In [ ]:
# import OS
import os
for x in os.listdir(folder_zips):
    if x.endswith(".zip"):
        print(x)
        x = os.path.join(folder_zips, x)
        with zipfile.ZipFile(x) as zf:
          # Get the folder name
          foldername = [
              info.filename for info in zf.infolist()
              if info.is_dir()][0]
          # Extract all the data
          zf.extractall()
         
        print(f'Extracted the files to {foldername}.')

In [ ]:
foldername

## Creating a folder list to iteratively compute TOA composite image for each of the LISS4 product

In [ ]:
folder = r"<-- INSERT PATH HERE -->"
import os
for x in os.listdir(folder_zips):
    if not x.endswith(".zip"):
        if not x.endswith(".docx"):
            print(x)

In [ ]:
folder_list = [r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->",
r"<-- INSERT PATH HERE -->"]

## Folder list with all the folder paths of training dataset for TOA composite creation

## TOA composite creation for every Liss4 test dataset

In [ ]:
import datetime
import ephem
import math
import os
import rioxarray as rxr
import xarray as xr
import zipfile

In [ ]:
from dask.distributed import Client, progress
client = Client() 
client

In [ ]:
def toa_conversion(folder):
    foldername = folder
    
    b2_path = os.path.join(foldername, 'BAND2.tif')
    b3_path = os.path.join(foldername, 'BAND3.tif')
    b4_path = os.path.join(foldername, 'BAND4.tif')
     
    b2_ds = rxr.open_rasterio(b2_path, chunks=True)
    b3_ds = rxr.open_rasterio(b3_path, chunks=True)
    b4_ds = rxr.open_rasterio(b4_path, chunks=True)
    
    metadata_filename = 'BAND_META.txt'
    metadata_filepath = os.path.join(foldername, metadata_filename)
     
    metadata = {}
    with open(metadata_filepath) as f:
      for line in f:
        key, value = line.split('=')
        metadata[key.strip()] = value.strip()
     
    scene_id = metadata['OTSProductID']
    print(f'Metadata extracted successfully for scene: {scene_id}')
    
    scene = xr.concat(
        [b2_ds, b3_ds, b4_ds], dim='band').assign_coords(
        band=['BAND2', 'BAND3', 'BAND4']
    )
     
    scene = scene.where(scene != 0)
    scene.name = scene_id
    # scene
    
    scene = xr.concat(
        [b2_ds, b3_ds, b4_ds], dim='band').assign_coords(
        band=['BAND2', 'BAND3', 'BAND4']
    )
     
    scene = scene.where(scene != 0)
    scene.name = scene_id
    # scene
    
    acq_date_str = metadata['DateOfPass']
    # Date is in the format 04-MAR-2023
    acq_date = datetime.datetime.strptime(
        acq_date_str, '%d-%b-%Y')
     
    sun_elevation_angle = metadata['SunElevationAtCenter']
    sun_zenith_angle = 90 - float(sun_elevation_angle)
    
    observer = ephem.Observer()
    observer.date = acq_date
    sun = ephem.Sun()
    sun.compute(observer)
    d = sun.earth_distance
    
    b2_sr = float(metadata['B2_Lmax'])
    b3_sr = float(metadata['B3_Lmax'])
    b4_sr = float(metadata['B4_Lmax'])
    
    sensor =  metadata['SatID']
    if sensor == 'IRS-R2':
        b2_esun = 185.33
        b3_esun = 157.766
        b4_esun = 111.359
    
    elif sensor == 'IRS-R2A':
        b2_esun = 185.347
        b3_esun = 158.262
        b4_esun = 110.81
    
    pi = math.pi
    sun_zenith_angle_rad = math.radians(sun_zenith_angle)
    
    b2_dn = scene.sel(band='BAND2')
    b3_dn = scene.sel(band='BAND3')
    b4_dn = scene.sel(band='BAND4')
    
    print(b2_sr)
    
    # print(b2_dn)
    b2_rad = b2_dn*b2_sr<-- INSERT PATH HERE -->
    b3_rad = b3_dn*b3_sr<-- INSERT PATH HERE -->
    b4_rad = b4_dn*b4_sr<-- INSERT PATH HERE -->
    
    b2_ref = (pi*b2_rad*d*d)/(b2_esun*math.cos(sun_zenith_angle_rad))
    b3_ref = (pi*b3_rad*d*d)/(b3_esun*math.cos(sun_zenith_angle_rad))
    b4_ref = (pi*b4_rad*d*d)/(b4_esun*math.cos(sun_zenith_angle_rad))
    
    reflectance_bands = [b2_ref, b3_ref, b4_ref]
    scene_ref = xr.concat(
        reflectance_bands, dim='band').assign_coords(
        band=['BAND2', 'BAND3', 'BAND4']
    ).chunk('auto')
    scene_ref.name = scene_id
    
    output_ds = scene_ref.to_dataset('band')
    
    output_file = f'test_{scene_id}.tif'
    
    output_file =os.path.join(foldername, output_file)
    output_options = {
        'driver': 'COG',
        'compress': 'deflate',
        'num_threads': 'all_cpus',
        'windowed': False 
    }
    #Save the raster.
    
    output_ds[['BAND2', 'BAND3', 'BAND4']].rio.to_raster(
        output_file, **output_options)
    print(f'Output file created {output_file}')

In [ ]:
for folder in folder_list:
    toa_conversion(folder)

## Saving the toa composite in the respective folders

# Classification of the TAO image to 0: No cloud, 1 : Cloud and 2: Shadow classes

## Saving accuracy assessment reports for all the test files:

### MLP

In [ ]:
import os
import torch
import torch.nn as nn
import joblib
import numpy as np
import geopandas as gpd
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

# --- Device Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# --- Paths ---
MODEL_PATH = r"<-- INSERT PATH HERE -->" 
SCALER_PATH = r"<-- INSERT PATH HERE -->"
CONFIG_PATH = r"<-- INSERT PATH HERE -->"
GEOJSON_DIR = r"<-- INSERT PATH HERE -->" # Directory with all the image test samples in geojson format
OUTPUT_CSV = os.path.join(GEOJSON_DIR, r"<-- INSERT PATH HERE -->")

# --- Load scaler and model config ---
scaler = joblib.load(SCALER_PATH)
model_config = joblib.load(CONFIG_PATH)

# --- Define Dataset ---
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# --- Define Model ---
class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )
    def forward(self, x):
        return self.net(x)

# --- Load Model ---
model = MLP(model_config["input_dim"], model_config["num_classes"]).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# --- Store Results ---
results = []

# --- Iterate Over GeoJSON Files ---
for file_name in os.listdir(GEOJSON_DIR):
    if file_name.endswith(".geojson"):
        file_path = os.path.join(GEOJSON_DIR, file_name)
        try:
            gdf = gpd.read_file(file_path)

            # Check required columns
            if 'Id' not in gdf.columns or not all(col in gdf.columns for col in ["SAMPLE_1", "SAMPLE_2", "SAMPLE_3"]):
                print(f"Skipping {file_name}: Required columns missing.")
                continue

            # Prepare features
            gdf['Id'] = gdf['Id'].astype(int)
            X = gdf[["SAMPLE_1", "SAMPLE_2", "SAMPLE_3"]].values.astype(np.float32)
            y = gdf["Id"].values.astype(np.int64)

            # Remove NaNs
            mask = ~np.isnan(X).any(axis=1)
            X, y = X[mask], y[mask]

            if len(y) == 0:
                print(f"Skipping {file_name}: No valid samples.")
                continue

            # Scale
            X_scaled = scaler.transform(X)

            # Create dataset and dataloader
            dataset = MyDataset(X_scaled, y)
            loader = DataLoader(dataset, batch_size=256, shuffle=False)

            # Predict
            y_true, y_pred = [], []
            with torch.no_grad():
                for X_batch, y_batch in loader:
                    X_batch = X_batch.to(device)
                    outputs = model(X_batch)
                    _, preds = outputs.max(1)
                    y_true.extend(y_batch.numpy())
                    y_pred.extend(preds.cpu().numpy())

            # Metrics
            acc = accuracy_score(y_true, y_pred)
            precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)

            results.append({
                "Filename": file_name,
                "Samples": len(y_true),
                "Accuracy": acc,
                "Precision": precision,
                "Recall": recall,
                "F1-Score": f1
            })

            print(f"Processed {file_name}: Accuracy = {acc:.4f}")

        except Exception as e:
            print(f"Error processing {file_name}: {e}")

# --- Save to CSV ---
results_df = pd.DataFrame(results)
results_df.to_csv(OUTPUT_CSV, index=False)
print(f"<-- INSERT PATH HERE -->✅ All reports saved to: {OUTPUT_CSV}")


## Combined accuracy metrics for the test dataset

In [ ]:
import os
import torch
import torch.nn as nn
import joblib
import numpy as np
import geopandas as gpd
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_recall_fscore_support

# --- Device Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# --- Paths ---
MODEL_PATH = r"<-- INSERT PATH HERE -->"
SCALER_PATH = r"<-- INSERT PATH HERE -->"
CONFIG_PATH = r"<-- INSERT PATH HERE -->"
GEOJSON_DIR = r"<-- INSERT PATH HERE -->" # directory with all the test image samples in geojson format

# --- Load scaler and model config ---
scaler = joblib.load(SCALER_PATH)
model_config = joblib.load(CONFIG_PATH)

# --- Define Dataset ---
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# --- Define Model ---
class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )
    def forward(self, x):
        return self.net(x)

# --- Load Model ---
model = MLP(model_config["input_dim"], model_config["num_classes"]).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# --- Combine All GeoJSONs ---
all_data = []
for file_name in os.listdir(GEOJSON_DIR):
    if file_name.endswith(".geojson"):
        file_path = os.path.join(GEOJSON_DIR, file_name)
        try:
            gdf = gpd.read_file(file_path)
            if 'Id' not in gdf.columns or not all(col in gdf.columns for col in ["SAMPLE_1", "SAMPLE_2", "SAMPLE_3"]):
                print(f"Skipping {file_name}: Required columns missing.")
                continue
            gdf['Id'] = gdf['Id'].astype(int)
            all_data.append(gdf)
        except Exception as e:
            print(f"Error reading {file_name}: {e}")

if not all_data:
    print("No valid data found.")
    exit()

# --- Combine into a single GeoDataFrame ---
combined_gdf = gpd.GeoDataFrame(pd.concat(all_data, ignore_index=True))

# --- Drop NaNs and extract features<-- INSERT PATH HERE --> ---
X = combined_gdf[["SAMPLE_1", "SAMPLE_2", "SAMPLE_3"]].values.astype(np.float32)
y = combined_gdf["Id"].values.astype(np.int64)

mask = ~np.isnan(X).any(axis=1)
X, y = X[mask], y[mask]

# --- Scale Features ---
X_scaled = scaler.transform(X)

# --- Dataset and Dataloader ---
dataset = MyDataset(X_scaled, y)
loader = DataLoader(dataset, batch_size=256, shuffle=False)

# --- Predict and Evaluate ---
y_true, y_pred = [], []

with torch.no_grad():
    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        _, preds = outputs.max(1)
        y_true.extend(y_batch.numpy())
        y_pred.extend(preds.cpu().numpy())

# --- Metrics ---
acc = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)
cm = confusion_matrix(y_true, y_pred)

# --- Print Report ---
print("<-- INSERT PATH HERE -->=== Combined Test Set Report ===")
print(f"Total Samples: {len(y_true)}")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

print("<-- INSERT PATH HERE -->=== Confusion Matrix ===")
print(cm)

print("<-- INSERT PATH HERE -->=== Classification Report ===")
print(classification_report(y_true, y_pred, digits=4))


## Classifying the images with the final model and saving them :

In [ ]:
import os
import torch
import torch.nn as nn
import joblib
import numpy as np
import rasterio
from torch.utils.data import DataLoader
from tqdm import tqdm

# Paths
MODEL_PATH = r"<-- INSERT PATH HERE -->"
SCALER_PATH = r"<-- INSERT PATH HERE -->"
CONFIG_PATH = r"<-- INSERT PATH HERE -->"
RASTER_DIR = r"<-- INSERT PATH HERE -->" # directory with all the folders of the TOA tif files
OUTPUT_DIR = r"<-- INSERT PATH HERE -->" # directory for saving the raster masks from classification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device in use: ', device)

# Load scaler and model config
scaler = joblib.load(SCALER_PATH)
model_config = joblib.load(CONFIG_PATH)

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )
    def forward(self, x):
        return self.net(x)

# Load model
model = MLP(model_config["input_dim"], model_config["num_classes"]).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# Process each scene folder
scene_folders = [f for f in os.listdir(RASTER_DIR) if os.path.isdir(os.path.join(RASTER_DIR, f))]

for scene in tqdm(scene_folders, desc="Processing Scenes"):
    folder_path = os.path.join(RASTER_DIR, scene)
    
    # Look for a tif file starting with "test_"
    tif_files = [f for f in os.listdir(folder_path) if f.startswith("test_") and f.endswith(".tif")]
    if not tif_files:
        print(f"❌ No test_*.tif found in {scene}")
        continue
    
    tif_path = os.path.join(folder_path, tif_files[0])  # Use first match

    try:
        with rasterio.open(tif_path) as src:
            img = src.read([1, 2, 3]).astype(np.float32)
            height, width = img.shape[1], img.shape[2]
            profile = src.profile

        flat_pixels = img.reshape(3, -1).T
        valid_mask = ~np.isnan(flat_pixels).any(axis=1)
        valid_pixels = flat_pixels[valid_mask]
        scaled_pixels = scaler.transform(valid_pixels)

        batch_size = 1024
        predictions = []
        with torch.no_grad():
            for i in range(0, len(scaled_pixels), batch_size):
                batch = torch.tensor(scaled_pixels[i:i+batch_size], dtype=torch.float32).to(device)
                outputs = model(batch)
                preds = torch.argmax(outputs, dim=1).cpu().numpy()
                predictions.append(preds)

        predicted_classes = np.full(flat_pixels.shape[0], fill_value=255, dtype=np.uint8)
        predicted_classes[valid_mask] = np.concatenate(predictions)
        classified_img = predicted_classes.reshape(height, width)

        # Save the classified raster with the scene folder name
        output_path = os.path.join(OUTPUT_DIR, f"{scene}.tif")
        profile.update(dtype=rasterio.uint8, count=1, nodata=255)
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(classified_img, 1)

    except Exception as e:
        print(f"❌ Error processing {scene}: {e}")

print(f"<-- INSERT PATH HERE -->✅ All classified outputs saved to: {OUTPUT_DIR}")


## conversion to shapefiles from tif masks

In [ ]:
import os
import geopandas as gpd
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape
import numpy as np
from tqdm import tqdm

# === Paths ===
RASTER_DIR = "<-- INSERT PATH HERE -->"
OUTPUT_SHP_DIR = "<-- INSERT PATH HERE -->"

os.makedirs(OUTPUT_SHP_DIR, exist_ok=True)

# === Process Each Raster ===
for file in tqdm(os.listdir(RASTER_DIR), desc="Vectorizing Rasters"):
    if file.endswith(".tif"):
        raster_path = os.path.join(RASTER_DIR, file)
        shp_output_path = os.path.join(OUTPUT_SHP_DIR, file.replace(".tif", ".shp"))

        with rasterio.open(raster_path) as src:
            image = src.read(1)  # Read single band
            mask = image != src.nodata  # Exclude NoData (255)

            results = (
                {"properties": {"Id": int(v)}, "geometry": s}
                for s, v in shapes(image, mask=mask, transform=src.transform)
            )

            geoms = list(results)

            if not geoms:
                print(f"⚠️ No valid shapes found in {file}")
                continue

            gdf = gpd.GeoDataFrame.from_features(geoms, crs=src.crs)

            # Optional: simplify or clean small polygons here

            gdf.to_file(shp_output_path)
            print(f"✅ Saved shapefile: {shp_output_path}")

print("<-- INSERT PATH HERE -->✅ All rasters vectorized to shapefiles.")
